# Работа с CustomTorch

Этот ноутбук представляет собой мануал по работе с библиотекой CusTorch

Данная библиотека является легковестным гибридом Keras и PyTorch и реализует базовые методы построения собственных нейронных сетей

## Начало работы

## Ознакомление с базовым функционалом

### Импорт
Для начала работы импортируем все классы и методы из библиотеки

In [2]:
from CustomTorch import *

Также импортируем уже знакомую библиоетку для работы с линейной алгеброй

In [3]:
import numpy as np

### класс Tensor

Данный класс является обёрткой для повторяющихся математическх операций над векорами, а также хранит промежуточные значения, такие как градиент

In [8]:
# Класс Tensor инициализируется через numpy.array
example_tensor_1 = Tensor(np.array([[1, 2], [3, 4]]))
example_tensor_2 = Tensor(np.array([[4, 3], [2, 1]]))

Мы можем производить различные операции над векторами

In [12]:
# Матричное умножение
new_tensor = example_tensor_1.__matmul__(example_tensor_2)
display(new_tensor.data)

# Сложение
new_tensor = example_tensor_1.__add__(example_tensor_2)
display(new_tensor.data)

array([[ 8,  5],
       [20, 13]])

array([[5, 5],
       [5, 5]])

## Класс NN

Расшифровывается как - Neural Network, то есть - нейронная сеть

Данный класс реализует обработку прохода данных по нейронной сети

In [28]:
model = NN()

В этот класс можно добавлять слои. 

Например, класс Linear реализует стандартный feed-forward слой с задаваемыми размерами входа и выхода

Также в библиотеке реализован класс Dropout. Подробнее о нём можете прочитать в документации и поэксперементировать сами

In [29]:
model.add(Linear(2, 16))
model.add(Linear(16, 16))
model.add(Linear(16, 2))

Пример получения выхода из нейронной сети

In [35]:
model.eval()
val_pred = model.forward(example_tensor_1)
val_pred.data

array([[0.66874939, 0.44595044],
       [0.66900018, 0.44612331]])

Также данный класс реализует кастомизируемые функцию потерь и оптимизатор для тренировки нейронной сети

In [32]:
loss_fn = MSELoss()
opt = SGD(model.parameters(), lr=1e-2)

И, разумеется, саму тренировку нейронной сети

In [34]:
model.train()
for epoch in range(5):
    y_pred = model.forward(example_tensor_1)
    loss = loss_fn.forward(y_pred, example_tensor_2)
    print(f"Loss:, {loss.data.item(0):.4}")
    
    # Обратное распространение
    grad_loss = loss_fn.backward(y_pred, example_tensor_2)
    model.backward(grad_loss)
    
    # Обновляем
    opt.step() 

Loss:, 5.342
Loss:, 5.255
Loss:, 5.17
Loss:, 5.087
Loss:, 5.005


## Собственные слои

Для создания собственного слоя наследуемся от базового класса Module

В качестве примера приведу написание линейного слоя с комментариями

In [ ]:
# Наследуемся от класса Module
class Linear(Module):
    # Т.к. слой линейный, то нам требуется размерность входа и выхода
    # Их мы запрашиваем в качестве аргументов при создании слоия
    def __init__(self, in_features: int, out_features: int):
        
        # Вызываем базовый конструктор 
        super().__init__()
        # Реализуем логику слоя. В данном случае это веса и bias
        self.weight = Tensor(np.random.randn(in_features, out_features) * 0.01)
        self.bias = Tensor(np.zeros(out_features))

    # Метод, который будет вызываться нейронной сетью при проходе
    def forward(self, x: Tensor) -> Tensor:
        self.input = x
        return Tensor(x.data @ self.weight.data + self.bias.data)

    # Обратное распространение ошибки
    # Возвращает изменённый градиент согласно Chain Rules (c)
    def backward(self, grad_output: Tensor):
        self.weight.grad = self.input.data.T @ grad_output.data
        self.bias.grad = np.sum(grad_output.data, axis=0)
        grad_input = grad_output.data @ self.weight.data.T
        return Tensor(grad_input)

    # Функция, возвращающая веса слоя оптимайзеру (необязательная)
    def parameters(self):
        return [self.weight, self.bias]